# Running this notebook

**The visualizations in this notebook will run in [jupyter lab](https://github.com/jupyterlab/jupyterlab#installation), not jupyter notebook. Google colab is not supported either. VS Code notebooks _might_ work but that has not been tested.** See the fastplotlib supported frameworks for more info: https://github.com/fastplotlib/fastplotlib/#supported-frameworks 

In [ ]:
from copy import deepcopy

import numpy as np
import pandas as pd
import tifffile
from ipywidgets import IntSlider, VBox
import fastplotlib as fpl

from caiman.motion_correction import high_pass_filter_space
from caiman.summary_images import correlation_pnr

import mesmerize_core as mc
from mesmerize_core.arrays import LazyTiff
from mesmerize_viz import *

In [ ]:
from mesmerize_core.caiman_extensions.cnmf import cnmf_cache

In [ ]:
if os.name == "nt":
    # disable the cache on windows, this will be automatic in a future version
    cnmf_cache.set_maxsize(0)

In [ ]:
# Mac users!
# temporary patch for Mac, won't be necessary in next release
# Thanks Ryan Ly for the PR! :D I need to dig into it more before merging
# conda_prefix_1_str = os.environ['CONDA_PREFIX'].replace(os.path.join(' ', 'envs', 'mescore')[1:], '')
# os.environ['CONDA_PREFIX_1'] = conda_prefix_1_str

In [ ]:
# This is just a pandas table display formatting option
pd.options.display.max_colwidth = 120

# Paths

`mesmerize-core` helps manage the outputs of caiman algorithms and organizes "parameter variants" - the output of a given combination of input data and algorithm parameters. In order to run the algorithms you must tell `mesmerize-core` where your _input data_ are located and decide on a **top level raw data directory**. For example consider the following directory structure of experimental data (you may organize your raw data however you wish, this is just an example). We can see that all the experimental data lies under `/data/group_name/my_name/exp_data`. Therefore we can use this `exp_data` dir as a `parent raw data path`. `mesmerize-core` will then only store the _relative_ paths to the raw data files, this allows you to move datasets between computers and filesystems. `mesmerize-core` does not store any hard file paths, only relative paths.

```
/data/group_name/my_name
                        └── exp_data
                            ├── axonal_imaging
                            │   ├── mouse_1
                            │   │   ├── exp_a.tiff
                            │   │   ├── exp_b.tiff
                            │   │   └── exp_c.tiff
                            │   ├── mouse_2
                            │   │   ├── exp_a.tiff
                            │   │   └── exp_b.tiff
                            │   └── mouse_3
                            └── hippocampus_imaging
                                ├── mouse_1
                                │   ├── exp_a.tiff
                                │   ├── exp_b.tiff
                                │   └── exp_c.tiff
                                ├── mouse_2
                                └── mouse_3
```

**For this demo set the `caiman_data` dir as the parent raw data path**

Sidenote: We recommend using [pathlib](https://docs.python.org/3/library/pathlib.html) instead of manually managing paths as strings. `pathlib` is just a part of the Python standard library, it makes it much easier to deal with paths and saves a lot of time in the long-run! It also makes your paths compatible across operating systems. Therefore even if you are on Windows you can use the regular `/` for paths, you do not have to worry about the strangeness of `\\` and `\`

In [ ]:
# for this demo set this dir as the path to your `caiman_data` dir
mc.set_parent_raw_data_path("/home/kushal/caiman_data/")

### Batch path, this is where caiman outputs will be organized

This can be anywhere, it does not need to be under the parent raw data path.

In [ ]:
batch_path = mc.get_parent_raw_data_path().joinpath("mesmerize-cnmfe/batch.pickle")

# Create a new batch

This creates a new pandas `DataFrame` with the columns that are necessary for mesmerize. In mesmerize we call this the **batch DataFrame**. You can add additional columns relevant to your experiment, but do not modify columns used by mesmerize.

Note that when you create a DataFrame you will need to use `load_batch()` to load it later. You cannot use `create_batch()` to overwrite an existing batch DataFrame

In [ ]:
# create a new batch
df = mc.create_batch(batch_path)
# to load existing batches use `load_batch()`
# df = load_batch(batch_path)

# View the dataframe

It is empty and has the required columns for mesmerize

In [ ]:
df

In [ ]:
# We'll use the data_endoscope movie from caiman
# download it if you don't have it
from caiman.utils.utils import download_demo
download_demo("data_endoscope.tif")

In [ ]:
movie_path = mc.get_parent_raw_data_path().joinpath("example_movies/data_endoscope.tif")

# gSig_filt

A high-pass spatial filter is useful for motion correction of miniscope 1p data, or other data which has large amounts of low frequency background flutuations.

The `gSig_filt` param sets the `sigma` of the gaussian kernel used for filtering. We can use fastplotlib to visualize the effects of this parameter. We want to remove the low frequency spatial information from the image to create better template images for motion correction.

Note that this is different from the `gSig` parameter used in CNMF!

In [ ]:
# get out input movie
# if it is memmapable you can use tifffile.memmap
# for other formats you can try LazyTiff, or any suitable lazy loader
input_movie = tifffile.imread(movie_path)

In [ ]:
# create a slider for gSig_filt
slider_gsig_filt = IntSlider(value=3, min=1, max=33, step=2,  description="gSig_filt")

def apply_filter(frame):
    # read slider value
    gSig_filt = (slider_gsig_filt.value, slider_gsig_filt.value)
    
    # apply filter
    return high_pass_filter_space(frame, gSig_filt)

# we can use frame_apply feature of `ImageWidget` to apply 
# the filter before displaying frames
funcs = {
    # data_index: function
    1: apply_filter  # filter shown on right plot, index 1
}

# input movie will be shown on left, filtered on right
iw_gs = fpl.ImageWidget(
    data=[input_movie, input_movie.copy()],
    frame_apply=funcs,
    names=["raw", "filtered"],
    grid_plot_kwargs={"size": (1200, 600)},
    cmap="gnuplot2"
)

def force_update(*args):
    # kinda hacky but forces the images to update 
    # when the gSig_filt slider is moved
    iw_gs.current_index = iw_gs.current_index
    iw_gs.reset_vmin_vmax()

iw_gs.reset_vmin_vmax()
    
slider_gsig_filt.observe(force_update, "value")

VBox([iw_gs.show(), slider_gsig_filt])

# reset vmin vmax when necessary!

# Motion correction parameters

Parameters for all algos have the following structure:

```python
{"main": {... params directly passed to caiman}}
```

In [ ]:
params =\
{
    "main":
    {
        "gSig_filt": (3, 3), # a gSig_filt value that brings out "landmarks" in the movie
        "pw_rigid": True,
        "max_shifts": (5, 5),
        "strides": (48, 48),
        "overlaps": (24, 24),
        "max_deviation_rigid": 3,
        "border_nan": "copy",
    }
}

# Add a "batch item", this is the combination of:
* algorithm to run, `algo`
* input movie to run the algorithm on, `input_movie_path`
* parameters for the specified algorithm, `params`
* a name for you to keep track of things, usually the same as the movie filename, `item_name`

In [ ]:
df.caiman.add_item(
    algo="mcorr",
    input_movie_path=movie_path,
    params=params,
    item_name=movie_path.stem
)

df

We can now see that there is one item, a.k.a. row or pandas `Series`, in the batch dataframe, we can add another item with the same input movie but with different parameters.

**When adding batch items with the same `input_movie_path` (i.e. same input movie but different parameters) it is useful to give them the same `item_name`.**

Let's try one more with different `gSig_filt`

In [ ]:
params2 =\
{
    "main":
    {
        "gSig_filt": (1, 1), # a gSig_filt value that brings out "landmarks" in the movie
        "pw_rigid": True,
        "max_shifts": (5, 5),
        "strides": (48, 48),
        "overlaps": (24, 24),
        "max_deviation_rigid": 3,
        "border_nan": "copy",
    }
}

df.caiman.add_item(
    algo="mcorr",
    input_movie_path=movie_path,
    params=params2,
    item_name=movie_path.stem
)

df

# We can see that there are two batch items for the same input movie.

Use a `for` loop to add multiple different parameter variants more efficiently.

In [ ]:
# copy the mcorr_params2 dict to make some changes
new_params = deepcopy(params)

# some variants of max_shifts
# but this can be any params
for shifts in [1, 3, 10]: 
    # deep copy is the safest way to copy dicts
    new_params = deepcopy(new_params)
    
    # assign the "max_shifts"
    new_params["main"]["max_shifts"] = (shifts, shifts)
    
    df.caiman.add_item(
      algo='mcorr',
      item_name=movie_path.stem,
      input_movie_path=movie_path,
      params=new_params
    )

In [ ]:
df

We can use the `caiman.get_params_diffs()` extension to see the unique parameters between rows with the same `item_name`

In [ ]:
diffs = df.caiman.get_params_diffs(algo="mcorr", item_name=df.iloc[0]["item_name"])
diffs

# Run an item

There is only one item in this DataFrame and it is located at index `0`. You can run a row using `df.iloc[index].caiman.run()`

Technical notes: On Linux & Mac it will run in subprocess but on Windows it will run in the local kernel. If using the subprocess backend (only Linux & Mac) you can use `run(wait=False)` if you don't want to block the kernel while running. 

In [ ]:
df.iloc[0].caiman.run()

# reload dataframe from disk when done
df = df.caiman.reload_from_disk()

# Run multiple batch items.

`df.iterrows()` iterates through rows and returns the numerical index and row for each iteration

In [ ]:
for i, row in df.iterrows():
    if row["outputs"] is not None: # item has already been run
        continue # skip
        
    process = row.caiman.run()
    
    # on Windows you MUST reload the batch dataframe after every iteration because it uses the `local` backend.
    # this is unnecessary on Linux & Mac
    # "DummyProcess" is used for local backend so this is automatic
    if process.__class__.__name__ == "DummyProcess":
        df = df.caiman.reload_from_disk()

# Reload the DataFrame

It is necessary to use `df = df.caiman.reload_from_disk()` after running a single batch item or a loop of batch items. You must not add new batch items until you reload it if you have ran items.

In [ ]:
df = df.caiman.reload_from_disk()

## We can see that the `outputs` column has been filled in

In [ ]:
df

# Visualize results

We will use the high pass spatial filter to make it easier to perform visual quality control.

In [ ]:
a = LazyTiff(df.iloc[0].caiman.get_input_movie_path())
a.shape

**This tiff file doesn't work with `tiffile.memmap` and `LazyTiff` is unable to guess its shape. So we will use `LazyTiff` with a forced shape**

In [ ]:
def load_weird_movie(path):
    return LazyTiff(path, shape=(1000, 128, 128))

In [ ]:
# high pass filter the data to see shifts more easily
filt = lambda x: high_pass_filter_space(x, (3, 3))

funcs = {
    0: filt,
    1: filt
}

viz_mcor = df.mcorr.viz(
    input_movie_kwargs={"reader": load_weird_movie},
    image_widget_kwargs={"frame_apply": funcs}
)

In [ ]:
viz_mcor.show()

# Customize the visualization

disable the spatial filter. Reset vmin vmax after removing it.

In [ ]:
viz_mcor.image_widget.frame_apply = dict()  # empty dict

Use the spatial filter again

In [ ]:
viz_mcor.image_widget.frame_apply = funcs

# Optional, cleanup

All the movies here look pretty good so I'll continue with `index = 0`. You can cleanup the DataFrame and remove all other items.

Remove batch items (i.e. rows) using `df.caiman.remove_item(<item_uuid>)`

**Note:** On windows calling `remove_item()` will raise a `PermissionError` if you have the memmap file open. The workaround is to shutdown the current kernel and then use `df.caiman.remove_item()`. For example, you can keep another notebook that you use just for cleaning unwanted mcorr items.

There is currently no way to close a `numpy.memmap`: https://github.com/numpy/numpy/issues/13510

In [ ]:
# make a list of rows we want to keep using the uuids
rows_keep = [df.iloc[3].uuid]
rows_keep

In [ ]:
for i, row in df.iterrows():
    if row.uuid not in rows_keep:
        df.caiman.remove_item(row.uuid)

df

# CNMF
## corr-pnr seeding

This visualization is to help determine values for `min_corr` (correlation) and `min_pnr` (peak to noise ratio) for seeding CNMFE. Pixels below these thresholds will be excluded from the results.

If `correlation_pnr` takes a long time you can increase the subsample to make it larger than `2`. Example: `mcorr_movie[::5]`

You should try different values of `gSig`, this is different from `gSig_filt`. You will use this gSig as a CNMFE param as well.

In [ ]:
# get the motion corrected output, this is a memmap array
mcorr_movie = df.iloc[0].mcorr.get_output()

gSig = 3
corr, pnr = correlation_pnr(mcorr_movie[::2], gSig=gSig, swap_dim=False)

In [ ]:
# to show the correlation and pnr images
iw_corr_pnr = fpl.ImageWidget(
    [corr, pnr], 
    names=["corr", "pnr"],
    grid_plot_kwargs={"size": (650, 300)},
    cmap="turbo",
)

# mcorr vids, we will display thresholded mcorr vids
mcorr_vids = [mcorr_movie.astype(np.float32) for i in range(4)]

# sync the threshold image widget with the corr-pnr plot
threshold_grid_plot_kwargs = {
    "controllers": [[iw_corr_pnr.gridplot["corr"].controller]*2]*2,
    "size": (650, 600)
}

iw_thres_movie = fpl.ImageWidget(
    mcorr_vids, 
    names=["over corr threshold", "over pnr threshold", "under corr threshold", "under pnr threshold"],
    # sync this with the corr-pnr plot
    grid_plot_kwargs=threshold_grid_plot_kwargs,
    cmap="gnuplot2"
)

# display threshold of the spatially filtered movie
def spatial_filter(frame):
    f = high_pass_filter_space(frame, (3, 3))
    return f


# threshold
def threshold(frame, mask):
    # optionally use spatial filter
    t = spatial_filter(frame)
    
    t = t.copy()
    
    t[mask] = t.min()
    
    return t

# Set the thresholded images using the vmin set from top subplots
# dict of threshold lambda wrappers to set on ImageWidget
# this sets the frame_apply for each subplot
threshold_funcs = {
    0: lambda frame: threshold(frame, corr < iw_corr_pnr.gridplot["corr"].graphics[0].cmap.vmin),
    1: lambda frame: threshold(frame, pnr < iw_corr_pnr.gridplot["pnr"].graphics[0].cmap.vmin),
    2: lambda frame: threshold(frame, corr > iw_corr_pnr.gridplot["corr"].graphics[0].cmap.vmin),
    3: lambda frame: threshold(frame, pnr > iw_corr_pnr.gridplot["pnr"].graphics[0].cmap.vmin)
}

# set the dict of lambda wrappers
iw_thres_movie.frame_apply = threshold_funcs

# update threshold plots when the corr pnr sliders move
def update_threshold_plots(*args):
    iw_thres_movie.current_index = iw_thres_movie.current_index

# this will get easier in the future
iw_corr_pnr.gridplot["corr"].docks["right"]["histogram_lut"].linear_region.selection.add_event_handler(update_threshold_plots)
iw_corr_pnr.gridplot["pnr"].docks["right"]["histogram_lut"].linear_region.selection.add_event_handler(update_threshold_plots)

VBox([iw_corr_pnr.show(), iw_thres_movie.show()])

# reset vmin vmax when necessary!!!

# corr and pnr values from the plot

In [ ]:
corr_pnr = {
    'min_corr': iw_corr_pnr.gridplot["corr"].graphics[0].cmap.vmin, # corr value from previous plot
    'min_pnr': iw_corr_pnr.gridplot["pnr"].graphics[0].cmap.vmin,  # PNR value from previous plot
}
corr_pnr

In [ ]:
params_cnmfe =\
{
    "main":
    {
        'method_init': 'corr_pnr',  # use this for 1 photon
        'K': None,
        'gSig': (gSig, gSig),
        'gSiz': (4 * gSig + 1, 4 * gSig + 1),
        'merge_thr': 0.7,
        'p': 1,
        'tsub': 2,
        'ssub': 1,
        'rf': 40,
        'stride': 20,
        'only_init': True,    # set it to True to run CNMF-E
        'nb': 0,
        'nb_patch': 0,
        'method_deconvolution': 'oasis',       # could use 'cvxpy' alternatively
        'low_rank_background': None,
        'update_background_components': True,  # sometimes setting to False improve the results
        'normalize_init': False,               # just leave as is
        'center_psf': True,                    # leave as is for 1 photon
        'ssub_B': 2,
        'ring_size_factor': 1.4,
        'del_duplicates': True,                # whether to remove duplicates from initialization
        **corr_pnr # unpack corr_pnr vals into here
    }
}

# Add a single cnmf item to the batch

When you use `algo="cnmfe"`, it basically forces the following parameters:
```python
"method_init": "corr_pnr",
"n_processes": n_processes,
"only_init": True,  # for 1p
"center_psf": True,  # for 1p
"normalize_init": False,  # for 1p
```

In [ ]:
df.caiman.add_item(
    algo="cnmfe",
    input_movie_path=df.iloc[0],
    params=params_cnmfe,
    item_name=df.iloc[0]["item_name"]
)

df

# Parameter search

Just like with motion correction, we can use loops to add multiple parameter variants. This is useful to perform a parameter search to find the params that work best for your dataset. Here I will use `itertools.product` which is better than deeply nested loops.

In [ ]:
from itertools import product

# variants of several parameters
# you can make lists for as many params as you want
K_variants = [None, 10]
merge_thr_variants = [0.6, 0.8, 0.9, 0.98]

# always use deepcopy like before
new_params_cnmf = deepcopy(params_cnmfe)

# create a parameter grid
# product is a nice way to create all combinations of multiple iterables like lists
parameter_grid = product(K_variants, merge_thr_variants)

# a single for loop to go through all the various parameter combinations
for K, merge_thr in parameter_grid:
    # deep copy params dict just like before
    new_params_cnmf = deepcopy(new_params_cnmf)
    
    # one set of parameter combinations
    new_params_cnmf["main"]["K"] = K
    new_params_cnmf["main"]["merge_thr"] = merge_thr
    
    # add param combination variant to batch
    df.caiman.add_item(
        algo="cnmfe",
        item_name=df.iloc[0]["item_name"],
        input_movie_path=df.iloc[0],
        params=new_params_cnmf
    )

See that there are a lot of new cnmf batch items

In [ ]:
df

# Param diffs

The index numbers on the diffs correspond to the indices in the parent DataFrame above

In [ ]:
df.caiman.get_params_diffs(algo="cnmfe", item_name=df.iloc[1]["item_name"])

# Run items

In [ ]:
for i, row in df.iterrows():
    if row["outputs"] is not None: # item has already been run
        continue # skip
        
    process = row.caiman.run()
    
    # on Windows you MUST reload the batch dataframe after every iteration because it uses the `local` backend.
    # this is unnecessary on Linux & Mac
    # "DummyProcess" is used for local backend so this is automatic
    if process.__class__.__name__ == "DummyProcess":
        df = df.caiman.reload_from_disk()

# We now have CNMFE outputs :D 

In [ ]:
df = df.caiman.reload_from_disk()
df

# Visualize using `mesmerize-viz`

In [ ]:
from mesmerize_viz import *

In [ ]:
viz = df.cnmf.viz(
    image_data_options=["input", "rcm"], # cnmfe does not support rcb and residuals yet
)

In [ ]:
viz.show()

# This seemingly complex visualization is still customizable!

Public attributes:

- `image_widget`: the `ImageWidget` in the visualization
- `plot_temporal`: the temporal `Plot`
- `plot_heatmap`: the heatmap `Plot`
- `cnmf_obj`: The cnmf object currently being visualized. This object gets saved to disk when you click the "Save Eval to disk" button.
- `component_index`: current component index, `int`

A few public methods:
- `show()` show the visualization
- `set_component_index(index: int)` manually set the component index

# Set frame_apply functions to the image widget. Reset the vmin-vmax as necessary.

In [ ]:
viz.image_widget

In [ ]:
funcs = {
    0: lambda frame: high_pass_filter_space(frame, (3, 3))
}

viz.image_widget.frame_apply = funcs

In [ ]:
viz.image_widget.cmap = "gray"